In [6]:
"""
scope: feature engineering for i2b2 data set
last modified: 11-20-19
author: andrew malinow
"""

'\nscope: feature engineering for i2b2 data set\nlast modified: 9/4/19\nauthor: andrew malinow\nto-do\n-fix blood pressure\n-finish vitals (temp, respitory)\n-have Jason create new columns for entities\n-Jason to double-check/revise as needed LOS calculation\n- word2vec on large text columns\n'

In [19]:
"""
install dependencies
"""
#!pip install pyforest
#!pip install -U spacy[cuda]
#!pip install -U cupy
#!pip install -U chainer
#!pip uninstall cupy -y
#!pip install cupy --no-cache-dir
!pip install nltk

     |################################| 1.5MB 4.1MB/s eta 0:00:01
  Stored in directory: /root/.cache/pip/wheels/96/86/f6/68ab24c23f207c0077381a5e3904b2815136b879538a24b483
Successfully built nltk


In [17]:
import spacy
spacy.require_gpu=False
if (spacy.require_gpu==True):
    print ('using gpu')
else:
    print ('not using GPU')

not using GPU


In [15]:
spacy.require_gpu=True
if (spacy.require_gpu==True):
    print ('using gpu')
else:
    print ('not using GPU')

using gpu


In [21]:
"""
Imports
"""
from pyforest import *
#import pickle
#import gensim
import cudf
import spacy
from spacy.lang.en import English
#from gensim.models import FastText
#import pandas as pd
#import cudf
#import os
#import re
import time
#import nltk
from nltk import sent_tokenize, word_tokenize
#from datetime import datetime
import re
import matplotlib.pyplot as plt





In [22]:
"""
global variables
"""
parser=English()
en_stop = set(nltk.corpus.stopwords.words('english'))

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/root/nltk_data'
    - '/conda/envs/rapids/nltk_data'
    - '/conda/envs/rapids/share/nltk_data'
    - '/conda/envs/rapids/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [23]:
"""
Read in data
"""

data=pd.read_csv("/rapids/notebooks/hostfs/Jason's Sandbox/jason_mimc-553_new.csv")
#print (data.head())
print (len(data))
#null_count=data.isna().sum()
#print (null_count)

205


In [296]:
#nltk.download('stopwords')
#nltk.download('wordnet')
#nltk.download('punkt')

In [17]:
"""
read all i2b2 files into a list
"""

path="/rapids/notebooks/hostfs/i2b2_data/"
files=[]

for (dirpath, dirnames, filenames) in os.walk(path):
    files += [os.path.join(dirpath, file) for file in filenames]
print(len(files))

701


In [298]:
###  Read in Drug Name file sourced from https://www.fda.gov/drugs/drug-approvals-and-databases/drugsfda-data-files

drugs=pd.read_csv(r"/rapids/notebooks/hostfs/Products.txt",sep="\t",error_bad_lines=False)
drug_list=list(set(drugs["DrugName"]))
drg=pd.DataFrame()
drg['drugs']=drug_list
drg.to_csv('drug_list.txt')

b'Skipping line 34962: expected 8 fields, saw 9\nSkipping line 34963: expected 8 fields, saw 9\nSkipping line 34964: expected 8 fields, saw 9\n'


In [313]:
"""
Open data, extract record id number and add to dataframe
"""
df=pd.DataFrame()
index=[]
record=[]
text=[]
target=[]
count=0

###create training set (n=300)
for file in files:
    index.append(str(count))
    with open (file) as f:
        x=(str(f.readline()).rstrip())
        x=x.split('RECORD')[1]
        x=re.split('#',x)[1]
        record.append(x)
        text.append(str(f.readlines()).replace('\n', ''))
    count+=1
df["index"]=index
df["record_id"]=record
df["text"]=text

In [300]:
"""
pre-processing Pandas: tokenize text
"""
tokens_in_record=[]
for record in df['text']:
    tokens=sent_tokenize(str(record))
    tokens_in_record.append(tokens)
df['tokens_in_record']=tokens_in_record

In [301]:
"""
feature engineering: pull out vitals
"""
all_vitals=[]
non_vital=[]
for record in df["text"]:
    
    missing=[]
    vitals=[]
    
    sentences=sent_tokenize(str(record))
    
    for line in sentences:
       
        if re.findall(r'temperature', str(line)): 
            junk, temp, keep=str(line).partition('temperature')
            vital=temp+keep
            vitals.append(vital)
            missing.append('na')
        if re.findall(r'blood pressure', str(line)):
            junk, bp, keep=str(line).partition('blood pressure')
            vital=bp+keep
            vitals.append(vital)
            missing.append('na')
        if re.findall(r'breathing',str(line)):
            junk, breath, keep=str(line).partition('breathing')
            vital=breath+keep
            vitals.append(vital)
            missing.append('na')
        if re.findall(r'respitory',str(line)):
            junk, breath, keep=str(line).partition('respitory')
            vital=breath+keep
            vitals.append(vital)
            missing.append('na')
        
        else:
            #vitals.append(line)
            continue
   
    all_vitals.append(vitals)
    non_vital.append(missing)
    continue
   
df['non-vitals']=non_vital
df['vitals']=all_vitals
df.to_csv('test_vitals.csv')


In [302]:
"""
Data Prep: clean strings
"""

df['vitals'] = df['vitals'].replace(r'\\n',' ', regex=True)
df['non-vitals'] = df['non-vitals'].replace(r'\\n',' ', regex=True)


In [303]:
"""
generate n-grams function
"""
def generate_ngrams(s, n):
    # Convert to lowercases
    s = s.lower()
    
    # Replace all none alphanumeric characters with spaces
    s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
    
    # Break sentence in the token, remove empty tokens
    tokens = [token for token in s.split(" ") if token != ""]
    tokens = [token for token in s.split(" ") if len(token)>=3]
    # Use the zip function to help us generate n-grams
    # Concatentate the tokens into ngrams and return
    ngrams = zip(*[tokens[i:] for i in range(n)])
    return [" ".join(ngram) for ngram in ngrams]

In [304]:
"""
generate n-grams for df['vitals'] to further refine vitals and prep for topic modelling
"""
ngrams=[]
for i, row in df.iterrows():
    vitals=row['vitals']
    clinical_ngrams=generate_ngrams(str(vitals),5)
    ngrams.append(clinical_ngrams)

df['clinical_ngrams']=ngrams
df['clinical_ngrams']

0      [blood pressure and this was, pressure and thi...
1      [temperature blood pressure 108 pulse, blood p...
2      [temperatures were not measured temperature, w...
3                                                     []
4                                                     []
5      [blood pressure room air temperature, pressure...
6                                                     []
7                                                     []
8      [breathing over past few weeks, over past few ...
9      [temperature was and heart rate, was and heart...
10     [blood pressure 196 jvp bilateral, pressure 19...
11                                                    []
12     [blood pressure 215 respiratory rate, pressure...
13                                                    []
14                                                    []
15     [breathing meals position nitro and, meals pos...
16     [temperatures and touch than the, and touch th...
17     [blood pressure improved

In [305]:
"""
feature engineering: pull out admission details
"""
admission_related=[]

for record in df["text"]:
    sentences=sent_tokenize(str(record))
    admit=[]
    for line in sentences:
        if re.findall(r'admission',str(line)):
            admit.append(line)
            continue
            
        elif re.findall(r'admitted',str(line)):
            admit.append(line)
            continue
        elif re.findall(r'admt',str(line)):
            admit.append(line)
            continue
        elif re.findall(r'DOA',str(line)):
            admit.append("DOA")
            continue   
        else:
            continue
    admission_related.append(admit)
df['admission_related']=admission_related

In [265]:
"""
generate n-grams for df['admission_related'] to further refine vitals and prep for topic modelling
"""
ngrams=[]
for i, row in df.iterrows():
    vitals=row['admission_related']
    clinical_ngrams=generate_ngrams(str(vitals),5)
    ngrams.append(clinical_ngrams)

df['admission_ngrams']=ngrams
df['admission_ngrams']

0      [two days prior admission the, days prior admi...
1      [sick sinus syndrome status post, sinus syndro...
2      [her massive edema was treated, massive edema ...
3      [ruled out ischemia and causes, out ischemia a...
4      [irreg beats abd surgical scars, beats abd sur...
5      [1995 work was negative with, work was negativ...
6                                                     []
7      [had been bedridden for several, been bedridde...
8      [the patient has had prior, patient has had pr...
9      [she had previously declined aortic, had previ...
10     [the patient was readmitted with, patient was ...
11     [presented day admission tni elevation, day ad...
12     [chest ray mild pulmonary vascular, ray mild p...
13     [she was admitted and treated, was admitted an...
14                                                    []
15     [none brief resume hospital course, brief resu...
16     [insertion pacemaker 2006 history present, pac...
17     [857012582 mmh 26868800 

In [266]:
df.columns

Index(['index', 'record_id', 'text', 'tokens_in_record', 'non-vitals',
       'vitals', 'clinical_ngrams', 'admission_related', 'admission_ngrams'],
      dtype='object')

In [267]:
"""
turn ngrams into single 'words' by replacing " " with "_"
"""
clinical_ngrams_concat=[]
for i, row in df.iterrows():
    ngram_list=[]
    ngrams=row['clinical_ngrams']
    for n in ngrams:
        n=str(n)
        a=str(n).replace(" ","_")
        ngram_list.append(a)
        continue
    clinical_ngrams_concat.append(ngram_list)
    continue
df['clinical_ngrams_concat']=clinical_ngrams_concat
df['clinical_ngrams_concat']

0      [blood_pressure_and_this_was, pressure_and_thi...
1      [temperature_blood_pressure_108_pulse, blood_p...
2      [temperatures_were_not_measured_temperature, w...
3                                                     []
4                                                     []
5      [blood_pressure_room_air_temperature, pressure...
6                                                     []
7                                                     []
8      [breathing_over_past_few_weeks, over_past_few_...
9      [temperature_was_and_heart_rate, was_and_heart...
10     [blood_pressure_196_jvp_bilateral, pressure_19...
11                                                    []
12     [blood_pressure_215_respiratory_rate, pressure...
13                                                    []
14                                                    []
15     [breathing_meals_position_nitro_and, meals_pos...
16     [temperatures_and_touch_than_the, and_touch_th...
17     [blood_pressure_improved

In [268]:
"""
pull out blood pressure from clinical ngrams
"""
blood_pressure=[]
bp=[]
for i,row in df.iterrows():
    blood_pressure=[]
    ngrams=row['ngrams_concat']
    if re.match(r'blood_pressure',str(ngrams)):
        if ''.join(c for c in str(ngrams) if c.isdigit()):
                a=str(row).replace("_"," ")
                # blood pressure 172 70
                if re.match(r"blood pressure \d{2,3} \d{2,3}",a):
                    blood_pressure.append(a)
                #blood pressure of 172 70
                elif re.match(r"blood pressure.* \d{2,3} \d{2,3}",a):
                    a=a.replace('blood pressure', "")
                    b=re.sub('[a-zA-Z]+'," ",a)
                    blood_pressure.append(b)         
                else:
                    blood_pressure.append(a)
                    
        else:
            continue
        bp.append(blood_pressure)
    else:
        bp.append(blood_pressure)
        i+=1
        continue
df['bp']=pd.Series(blood_pressure)
df['bp']

KeyError: 'ngrams_concat'

In [ ]:
df.to_csv('test.csv')

In [269]:
df.columns

Index(['index', 'record_id', 'text', 'tokens_in_record', 'non-vitals',
       'vitals', 'clinical_ngrams', 'admission_related', 'admission_ngrams',
       'clinical_ngrams_concat'],
      dtype='object')

In [270]:
"""
pull out temp, pulse and blood pressure from clinical ngrams
"""
bp=[]
temperature=[]
px=[]
for i , row in df.iterrows():
    blood_pressure=[]
    temp=[]
    pulse=[]
    ngrams=row['vitals']
  
    #test=row['tokens_in_record']
    
    for n in ngrams:
        if re.match(r"(\bblood pressure)(.*[0-9]\b)",str(n)):
            match_b=re.search(r"(\bblood pressure)(.*[0-9]\b)",str(n)).group()
            blood_pressure.append(match_b)
            
        
        #if re.match(r"(\btemperature)(.*[0-9]\b)",str(n)):
        try:    
            match_t=re.match(r"(\btemperature)(.*[0-9]\b)",str(n)).group()
            temp.append(match_t)
        except AttributeError as a:
            continue
        #if re.match(r"(\bpulse)(.*[0-9])",str(n)):
        try:
            match_p=re.match(r"(pulse)(.*[0-9]\b)",str(n)).group()
            pulse.append(match_p)
        except AttributeError as a:
            continue
        #else:
            continue
   
        #continue

    bp.append(blood_pressure)
    temperature.append(temp)
    px.append(pulse)
  
df['blood_pressure']=pd.Series(bp)
df['pulse']=pd.Series(px)
df['temperature']=pd.Series(temperature)


In [272]:
count_nan = len(df) - df.count()
print (count_nan)

index                     0
record_id                 0
text                      0
tokens_in_record          0
non-vitals                0
vitals                    0
clinical_ngrams           0
admission_related         0
admission_ngrams          0
clinical_ngrams_concat    0
blood_pressure            0
pulse                     0
temperature               0
dtype: int64


In [273]:
###iterate through tokens and split record
admit_related=[]
discharge_related=[]

for single_record in text:
    #single_record_list=sent_tokenize(str(single_record))
    admission, word, discharge=str(single_record).partition("HOSPITAL COURSE:")
    admit_related.append(admission)
    garbage,w,keep=str(admission).partition("DISPOSITION")
    discharge_related.append(keep)
    continue

df['admit_related']=pd.Series(admit_related)
df['discharge_related']=pd.Series(discharge_related)


In [275]:
feature engineering:medications
"""

###  Iterate through text to find matches

medications=[]

for record in df['text']:
    med_list=[]
    for drug in drug_list:
        if str(drug) in str(record):
            med_list.append(drug)         
            continue
    medications.append(med_list)
    continue
df['all_meds']=medications
#df.to_csv('test-all-meds.csv')

In [276]:
"""
feature engineering: admission medications
"""
meds_on_admission=[]
for record in df["text"]:
    sentences=sent_tokenize(str(record))
    for line in sentences:
        if re.search(r'MEDICATIONS', str(line)) and 'ADMISSION' in str(line):
            try:
                x=str(line).split('ADMISSION:')
                x=x[1]
                meds_on_admission.append(x)
                continue
            except IndexError as e:
                continue
        else:
            continue
   
df['meds_on_admission']=pd.Series(meds_on_admission)
        

In [277]:
"""
feature engineering: discharge medications
"""
discharge_related=[]
for record in df["text"]:
    sentences=sent_tokenize(str(record))
    for line in sentences:
        if re.search(r'MEDICATIONS', str(line)) and 'DISCHARGE' in str(line):
            try:
                x=str(line).split('DISCHARGE MEDICATIONS:')
                x=x[0]
                discharge_related.append(x)
                continue
        
            except IndexError as e:
                print (e)
                continue
        else:
            continue 
df['discharge_related']=pd.Series(discharge_related)

In [278]:
"""
feature-engineering: discharge medication context
"""
discharge_meds=[]
discharge_medication_context=[]
for single_record in df["discharge_related"]:
    dis_meds=[]
    dis_med_cont=[]
    sentences=sent_tokenize(str(single_record))
    for sentence in sentences:
        for drug in drug_list:
            if str(drug) in str(sentence):
                    dis_med_cont.append(sentence)
                    dis_meds.append(drug)
                    continue
            else:
                    continue
        continue
    discharge_meds.append(dis_meds)
    discharge_medication_context.append(dis_med_cont)
    
df['discharge_medication_context']=discharge_medication_context
df['discharge_medications']=discharge_meds

In [ ]:
#df.to_csv('test-discharge-meds.csv')

In [326]:
"""
feature engineering:principal diagnosis
"""
principal_dx=[]
for record in text:
    try:
        d_dx=re.search(("PRINCIPAL DIAGNOSIS:.*$"), str(record))
        d_dx=str(d_dx).split(":")
        d_dx=d_dx[1]
        d_dx=str(d_dx).split(".")
        d=d_dx[0]
        d=re.sub(r'([^\s\w]|_)+', '', d)
        principal_dx.append(d)
        continue
    except IndexError as e:
        d='n/a'
        principal_dx.append(d)
        continue

In [328]:
df['dx']=pd.Series(principal_dx)

In [330]:
"""
find similar medications based on vectorizing admission related
text in the record
"""
from nltk import sent_tokenize,word_tokenize
from gensim.models import Word2Vec
text=re.sub(r'([^\s\w]|_)+', '', str(text))

sentences=word_tokenize(str(text))
model = Word2Vec([sentences], min_count=1)
#vector='infection'
#model.similar_by_vector(vector, topn=10, restrict_vocab=None)
#model.most_similar_to_given(entity1, entities_list)

In [365]:
col=[]
context=[]
for row in df['text']:
    
    if re.search('infection',str(row)):
        col.append(model.wv.similar_by_word('infection'))
        context.append(row)
        continue
    else:
        col.append(0)
        context.append(0)
        continue
df['col']=col
df['context']=context
df.to_csv('infection.csv')

In [370]:
rel=[]
for row in df['text']:
    
    if re.findall('YOUKER',str(row)):
        rel.append(model.wv.similar_by_word('YOUKER'))
        
        continue
    else:
        rel.append(0)
        
        continue
df['rel']=rel
df.to_csv('infection2.csv')

In [358]:
df2=pd.DataFrame()
related=[]
med_list=[]
for med in drug_list:
    med_list.append(med)
    try:
        related.append(model.wv.similar_by_word(str(med)))
        continue
    except KeyError as a:
        related.append('n/a')
        continue
df2['medication']=med_list
df2['related']=related
df2.dropna()
df2.to_csv("med_use.csv")

In [281]:
"""
feature engineering: admission date
"""
admission_dt=[]
#text={'Signed DIS Admission Date: 08/28/06 Report Status: Signed'}
for record in text:
     try:
        x= (re.search(("Admission Date:.*$"), str(record)))
        x=str(x).split(r'\\n')[0]
        x=str(x).split(":")[1]
        x=str(x).split(" ")[1]
        
        if x[0]=='2':
            x= 'NaT'
            admission_dt.append(x)
            continue
        else:
            admission_dt.append(x)
     except AttributeError as e:
        continue
df['admission_dt']=admission_dt



In [282]:
"""
feature engineering: discharge date
"""
discharge_dt=[]
for record in text:
     try:
        x= (re.search(("Discharge Date:.*$"), str(record)))
        x=str(x).split(r'\\n')[0]
        x=str(x).split(":")[1]
        if x[0]==0 or x[2]==0:
            x= 'NaT'
            discharge_dt.append(x)
            continue
        else:
            discharge_dt.append(x)
     except AttributeError as e:
            x= 'NaT'
            discharge_dt.append(x)
            continue
     except IndexError as e:
            x= 'NaT'
            discharge_dt.append(x)
            continue
df['discharge_dt']=discharge_dt

In [283]:
"""
feature engineering:deceased flag
"""
deceased_flag=[]

for cell in df['discharge_dt']:
    if  'NaT' in str(cell):
        deceased_flag.append('1')
        continue
    
    if re.findall(r'pronounced at',str(line),re.I) or re.search(r'pronounced dead',str(line),re.I):
        
        deceased_flag.append('1')
        continue
    else:
        deceased_flag.append('0')
        continue

In [284]:
"""
feature engineering:medications
"""
medications=[]
for record in text:
    try:
        d_dx=re.findall(("MEDICATIONS:.*$"), str(record))
        d_dx=str(d_dx).split(":")
        d_dx=d_dx[1]
        d_dx=str(d_dx).split(".")
        d=d_dx[0]
        d=re.sub(r'([^\s\w]|_)+', '', d)
        medications.append(d)
        continue
    except IndexError as e:
        d='n/a'
        medications.append(d)
        continue
df['medications']=medications

In [285]:
"""
feature engineering:medications 2
"""
meds=[]
for record in text:
    for word in word_tokenize(str(record)):
        word=re.sub(r'([^\s\w]|_)+', '', str(word))
        if word in drug_list:
            meds.append(word)
            continue
        else:
            continue
                           
df['meds']=pd.Series(meds)

In [ ]:
"""
feature engineering: quantified clinical phrases
"""
clinical_markers=[]
for record in text:
    sentences=sent_tokenize(str(record))
    for line in sentences:
        if re.findall(r'[A-Za-z0-9]', str(line)):
            clinical_markers.append(line)
            continue
        else:
            continue

In [286]:
"""
feature engineering-derived: length of stay
"""
df['discharge_dt']=pd.to_datetime(df['discharge_dt'],errors='coerce')
df['admission_dt']=pd.to_datetime(df['admission_dt'],errors='coerce')
df['discharge_dt'] = df['discharge_dt'].dt.date
df['admission_dt'] = df['admission_dt'].dt.date

df['length_of_stay']=df['discharge_dt']-df['admission_dt']
df['length_of_stay']=abs(df['length_of_stay'])
###formatting los and conveting to int
los=[]
for value in df['length_of_stay']:
    try:
        value=str(value).split(" ")
        value=int(value[0])
        los.append(value)
    except ValueError as e:
        pass
df['los']=pd.Series(los)
df['length_of_stay']=df['los']       

In [ ]:
"""
add features to orginal df
"""
df['medications']=pd.Series(medications)
df["admission_dt"]=admission_dt
df["discharge_dt"]=discharge_dt
df['principal_dx']=principal_dx
df['discharge_related']=pd.Series(discharge_related)
df['meds_on_admission']=pd.Series(meds_on_admission)
df['deceased_flag']=deceased_flag
df['clinical_markers']=pd.Series(clinical_markers)
df['length_of_stay']=df['los']

In [287]:
"""
write parsed data to output file
"""
os.chdir(r"/rapids/notebooks/hostfs/DX-Predictor-Ouput")
df.to_csv('i2b2-parsed-9-13-19.csv')

In [289]:
df['length_of_stay']

0      120.0
1      150.0
2       32.0
3      141.0
4       10.0
5      259.0
6      462.0
7      208.0
8      297.0
9       92.0
10      85.0
11     177.0
12      80.0
13      23.0
14     291.0
15      13.0
16     315.0
17     145.0
18     127.0
19     261.0
20     240.0
21      57.0
22      57.0
23      87.0
24      90.0
25      40.0
26      76.0
27     128.0
28      20.0
29      38.0
       ...  
669      NaN
670      NaN
671      NaN
672      NaN
673      NaN
674      NaN
675      NaN
676      NaN
677      NaN
678      NaN
679      NaN
680      NaN
681      NaN
682      NaN
683      NaN
684      NaN
685      NaN
686      NaN
687      NaN
688      NaN
689      NaN
690      NaN
691      NaN
692      NaN
693      NaN
694      NaN
695      NaN
696      NaN
697      NaN
698      NaN
Name: length_of_stay, Length: 699, dtype: float64

In [ ]:
"""
feature engineering: use LDA to create additional features from unstructured fields
"""

In [ ]:
"""
pre-processing: tokenize text function using Spacy
"""
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [ ]:
"""
pre-processing: prepare text for topic modelling
"""
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 2]
    tokens = [token for token in tokens if token not in en_stop]
    
    return tokens

In [ ]:
"""
pre-processing: tokenize clinical markers text
"""
clinical_markers_tokens=[]
for n in df['clinical_markers']: 
    clinical_markers_tokens.append(prepare_text_for_lda(str(n)))
df['tokens']=clinical_markers_tokens


In [ ]:
"""
pre-processing: tokenize discharge_related text
"""
discharge_related_tokens=[]
for n in df['discharge_related']: 
    discharge_related_tokens.append(prepare_text_for_lda(str(n)))
df['discharge_related_tokens']=discharge_related_tokens


In [ ]:
"""
pre-processing: tokenize 'ngrams_concat' for topic modelling
"""
ngrams_concat_tokens=[]
for n in df['ngrams_concat']: 
    ngrams_concat_tokens.append(prepare_text_for_lda(str(n)))
df['ngrams_concat_tokens']=ngrams_concat_tokens


In [ ]:
"""
pre-processing: tokenize 'admission_ngrams_concat' for topic modelling
"""
admission_ngrams_concat_tokens=[]
for n in df['admission_ngrams_concat']: 
    admission_ngrams_concat_tokens.append(prepare_text_for_lda(str(n)))
df['admission_ngrams_concat_tokens']=admission_ngrams_concat_tokens


In [ ]:
"""
pre-processing: tokenize discharge_medication_context text
"""
discharge_medication_context_tokens=[]
for n in df['discharge_medication_context']: 
    discharge_medication_context_tokens.append(prepare_text_for_lda(str(n)))
df['discharge_medication_context_tokens']=discharge_medication_context_tokens


In [ ]:
"""
pre-processing: tokenize df['admission_related'] text
"""
admission_related_tokens=[]
for n in df['admission_related']: 
    admission_related_tokens.append(prepare_text_for_lda(str(n)))
df['admission_related_tokens']=admission_related_tokens


In [ ]:
df.columns

In [ ]:
"""
pre-processing: tokenize medications
"""
medication_tokens=[]
for n in df['medications']: 
    medication_tokens.append(prepare_text_for_lda(str(n)))
df['medication_tokens']=medication_tokens


In [ ]:
"""
pre-processing: tokenize meds_on_admission
"""
meds_on_admission_tokens=[]
for n in df['meds_on_admission']: 
    meds_on_admission_tokens.append(prepare_text_for_lda(str(n)))
df['meds_on_admission_tokens']=meds_on_admission_tokens


In [ ]:
"""
pre-processing: tokenize all_vitals
"""
all_vitals_tokens=[]
for n in df['vitals']: 
    all_vitals_tokens.append(prepare_text_for_lda(str(n)))
df['all_vitals_tokens']=all_vitals_tokens


In [ ]:
"""
feature engineering-derived: use LDA on clinical markers, medications tokens
"""

In [ ]:
import os
cwd = os.getcwd()
print(cwd)
df.columns

In [ ]:
df['admission_ngrams_concat_tokens']

In [ ]:
"""
feature engineering-derived: use LDA on ngrams_concat
create dictionary and corpus and save for future use
"""
start=time.time()
dictionary = gensim.corpora.Dictionary(df['admission_ngrams_concat_tokens'])

#create corpus 
corpus = [dictionary.doc2bow(text) for text in admission_ngrams_concat_tokens]
#save corpus and dictionary
pickle.dump(corpus, open('i2b2-corpus.pkl', 'wb'))
dictionary.save('i2b2-admission-ngrams_concat.gensim')
end=time.time()
print (end-start)

In [ ]:
df['admission_related']

In [ ]:
df['clinical_ngrams']

In [ ]:
"""
feature engineering-derived: use LDA on all_vitals
create dictionary and corpus and save for future use
"""
start=time.time()
dictionary = gensim.corpora.Dictionary(df['admission_related_tokens'])

#create corpus 
corpus = [dictionary.doc2bow(text) for text in admission_related_tokens]
#save corpus and dictionary
pickle.dump(corpus, open('i2b2-corpus.pkl', 'wb'))
dictionary.save('i2b2-admission_related.gensim')
end=time.time()
print (end-start)

In [ ]:
"""
feature engineering-derived: use LDA on clinical ngrams concat
create dictionary and corpus and save for future use
"""
start=time.time()
dictionary = gensim.corpora.Dictionary(df['clinical_ngrams_concat'])

#create corpus 
corpus = [dictionary.doc2bow(text) for text in clinical_ngrams_concat]
#save corpus and dictionary
pickle.dump(corpus, open('i2b2-corpus.pkl', 'wb'))
dictionary.save('i2b2-admission_related.gensim')
end=time.time()
print (end-start)

In [ ]:
"""
train and save an LDA model
use num_topics parameter to determine the number of topics for the model, and num_words parameter for how much to show
"""
from gensim import models
start=time.time()
#model=models.KeyedVectors.load('ft-i2b2-model.gensim')
lda=gensim.models.LdaMulticore(corpus=corpus,num_topics=4,id2word=dictionary,passes=100,workers=7)
topics=lda.print_topics(num_words=2)
end=time.time()
print (end-start)

In [ ]:
topics=lda.print_topics(num_words=5)
for topic in topics:
    print (topic)

In [371]:
"""
visualize results
"""
import pyLDAvis
import pyLDAvis.gensim 
import matplotlib.pyplot as plt
%matplotlib inline
# Visualize the topics
id2word=dictionary
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda, corpus, id2word)
vis

/conda/envs/rapids/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.091185  0.054681       1        1  26.132710
0     -0.105698  0.043279       2        1  25.457747
1      0.005069 -0.117372       3        1  24.631176
3      0.009444  0.019412       4        1  23.778366, topic_info=     Category      Freq                                           Term  \
term                                                                     
4719  Default  3.000000          give_units_subcutaneously_blood_sugar   
3806  Default  3.000000     saturation_room_air_cardiovascular_regular   
3805  Default  3.000000           room_air_cardiovascular_regular_rate   
3775  Default  3.000000            air_cardiovascular_regular_rate_and   
996   Default  4.000000         respiratory_rate_and_oxygen_saturation   
3802  Default  4.000000                   regular_rate_and_rhythm_with   
2224  Default  4.000000         cardiovascular_regular_rate_and_rhythm   
4804  Default  3.000000           then_give_units_subcutaneously_blood   
2149  Default  3.000000         respiratory_rate_oxygen_saturation_100   
298   Default  5.000000               oxygen_saturation_room_air_blood   
2050  Default  4.000000            blood_pressure_130_respiratory_rate   
926   Default  4.000000            blood_pressure_140_respiratory_rate   
5284  Default  2.000000                rate_oxygen_saturation_100_room   
3797  Default  2.000000      oxygen_saturation_room_air_cardiovascular   
3789  Default  2.000000                   heart_rate_and_regular_blood   
3799  Default  2.000000                rate_and_regular_blood_pressure   
3801  Default  2.000000               regular_blood_pressure_right_arm   
3808  Default  2.000000             temperature_heart_rate_and_regular   
3778  Default  2.000000               and_regular_blood_pressure_right   
228   Default  7.000000        respiratory_rate_oxygen_saturation_room   
227   Default  7.000000                rate_oxygen_saturation_room_air   
2669  Default  2.000000            blood_pressure_120_respiratory_rate   
2054  Default  3.000000                 oxygen_saturation_100_room_air   
855   Default  3.000000                  blood_pressure_and_heart_rate   
170   Default  6.000000                   blood_pressure_the_right_arm   
296   Default  8.000000                   blood_pressure_less_than_100   
5269  Default  3.000000            blood_pressure_160_respiratory_rate   
869   Default  2.000000         respiratory_rate_and_saturating_liters   
3279  Default  2.000000                   mouth_every_hours_needed_for   
1341  Default  2.000000             breathing_with_room_air_saturation   
...       ...       ...                                            ...   
5937   Topic4  1.163156         air_general_appearance_elderly_looking   
5939   Topic4  1.163156  appearance_elderly_looking_gentleman_apparent   
5938   Topic4  1.163156       apparent_distress_with_bilateral_hearing   
2054   Topic4  2.198228                 oxygen_saturation_100_room_air   
855    Topic4  2.197276                  blood_pressure_and_heart_rate   
2050   Topic4  2.717641            blood_pressure_130_respiratory_rate   
926    Topic4  2.714675            blood_pressure_140_respiratory_rate   
531    Topic4  1.682347           temperature_pulse_blood_pressure_140   
1162   Topic4  1.680294                   pressure_less_than_100_heart   
633    Topic4  2.199095            liters_nasal_cannula_blood_pressure   
296    Topic4  2.724993                   blood_pressure_less_than_100   
323    Topic4  1.163155              pressure_115_respiratory_rate_and   
325    Topic4  1.163155             rate_and_saturations_liters_oxygen   
327    Topic4  1.163155        respiratory_rate_and_saturations_liters   
328    Topic4  1.163155          saturations_liters_oxygen_with_liters   
1106   Topic4  1.682715            and_respiratory_rate_blood_pressure   
308    

In [ ]:
"""
feature engineering-derived: use LDA on medications tokens
create dictionary and corpus and save for future use
"""
start=time.time()
dictionary = gensim.corpora.Dictionary(meds_tokens)

#create corpus 
corpus = [dictionary.doc2bow(text) for text in meds_tokens]
#save corpus and dictionary
pickle.dump(corpus, open('i2b2-corpus.pkl', 'wb'))
dictionary.save('i2b2-medications.gensim')
end=time.time()
print (end-start)

In [ ]:
"""
feature engineering-derived: use LDA on full text
create dictionary and corpus and save for future use
"""
dictionary = gensim.corpora.Dictionary(text_tokens)

#create corpus 
corpus = [dictionary.doc2bow(text) for text in text_tokens]
#save corpus and dictionary
pickle.dump(corpus, open('i2b2-corpus.pkl', 'wb'))
dictionary.save('i2b2-medications.gensim')


In [ ]:
"""
feature-engineering: bi-grams and tri-grams
data: df['discharge_medication_context']
"""
from nltk import ngrams
from collections import Counter

In [ ]:
n = 4
#bigrm = list(nltk.bigrams((str(df['discharge_medication_context']).split())))
fourgrams=[]
for line in df['discharge_medication_context']:
    tokens=word_tokenize(str(line))
    trigrams=ngrams(tokens,3)


In [ ]:
df['discharge_medication_context_bigrams']

In [ ]:
""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
###EXPERIMENTAL"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

In [ ]:
"""
Finding diagnosis related content by using similarity matching
need to try Facebook Research's GPU accelerated library:https://github.com/facebookresearch/faiss
https://github.com/facebookresearch/faiss/issues/54
"""

In [ ]:
from difflib import SequenceMatcher
from fuzzywuzzy import fuzz
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [ ]:
df1=pd.DataFrame()
start=time.time()
lines=[]
matches=[]
diagnosis_list=[]
text=[]
diagnosis_list.append(sent_tokenize(str(dx_list)))
matches_list=[]
text_list=[]
for row in df['text']:
    matches=[]
    sentences=[]
    lines.append(sent_tokenize(str(row)))
    
    for line in lines:
        sentences.append(sent_tokenize(str(line)))
        for sentence in sentences:
            for a in sentence:
                a=re.sub(r'([^\s\w]|_)+', '', str(a))
                a=result = ''.join([i for i in a if not i.isdigit()])
                for n in dx_list:              
                   
                    #base,blank,modifier=str(n).partition(" ")
                    
                    b=re.sub(r'([^\s\w]|_)+', '', str(n))
                    #if str(b) in str(a):
                   
                    
                    value=fuzz.token_set_ratio(a,b)
                    if value>70:
                        keep, word,snippet=a.partition(str(b))
                        matches.append(b)
                        text.append(a)
                        continue
                    else:
                        #matches.append("none")
                        continue
            continue
        continue
    
    matches_list.extend(matches)
    text_list.extend(text)
    continue
end=time.time()
print("all records processed") 
print("time to process:",(end-start))
list(set(matches))
df1['diagnostic']=matches
df1['text']=text


In [ ]:
###  Read in Diagnosis file 
#take only first word"
dx=pd.read_csv(r"/rapids/notebooks/hostfs/dx.txt",sep="\t",error_bad_lines=False)
dx_list=dx.values.tolist()
single_dx_list=[]
for i in dx_list:
    i=str(i).split()[0].lower()
    single_dx_list.append(i)
list(set(single_dx_list))

In [ ]:
df